<a href="https://colab.research.google.com/github/bhattacharjee/msc-ai-project/blob/main/DeepANN/deep_ann_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
USE_FLOAT32 = True
REDUCE_DF_SIZE = True


import tensorflow as tf
from keras.utils import np_utils
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import logging, sys, random, glob
from google.colab import drive
from functools import lru_cache
import sklearn
from sklearn.model_selection import train_test_split

from tensorflow import keras as K
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
import IPython

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

import gc


drive.mount("/content/drive")

def set_random_seed():
    np.random.seed(1)
    random.seed(1)
    tf.random.set_seed(1)

root = logging.getLogger()
root.setLevel(logging.INFO)
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

!if [[ ! -d /content/drive/MyDrive/MSCPROJDATA ]]; then cp -r /content/drive/MyDrive/MSCPROJDATA .; fi

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset_names = ["baseline", "advanced-only", "fourier-only", "baseline-and-fourier", "advanced-and-fourier", "advanced", "fourier"]
def get_columns(thisdf):
    baseline_columns = [c for c in thisdf.columns if c.startswith('baseline') and "head" not in c and "tail" not in c]
    baseline_columns = [c for c in baseline_columns if "filesize" not in c]
    baseline_columns = [c for c in baseline_columns if "begin" not in c and "end" not in c]

    advanced_columns = [c for c in thisdf.columns if "advanced" in c]
    advanced_columns = [c for c in advanced_columns if "begin" not in c and "end" not in c]
    advanced_columns = [c for c in advanced_columns if "head" not in c and "tail" not in c]
    advanced_columns = [c for c in advanced_columns if "start" not in c]
    advanced_columns_only = list(set(advanced_columns))
    advanced_columns = list(set(advanced_columns + baseline_columns))

    fourier_columns = [c for c in thisdf.columns if "fourier" in c and "value" not in c]
    fourier_columns = [c for c in fourier_columns if "1byte" in c]
    fourier_columns = [c for c in fourier_columns if "begin" not in c and "end" not in c]
    fourier_columns = [c for c in fourier_columns if "head" not in c and "tail" not in c]
    fourier_columns = [c for c in fourier_columns if "start" not in c]
    fourier_columns_only = list(set(fourier_columns))
    fourier_columns = list(set(advanced_columns + fourier_columns))
    
    baseline_and_advanced = list(set(baseline_columns + advanced_columns_only))
    baseline_and_fourier = list(set(baseline_columns + fourier_columns_only))
    advanced_and_fourier = list(set(advanced_columns_only + fourier_columns_only))

    all_columns = [c for c in thisdf.columns]
    
    return {\
        "baseline": baseline_columns,\
        "advanced-only": advanced_columns_only,\
        "fourier-only": fourier_columns_only,\
        "baseline-and-fourier": baseline_and_fourier,\
        "advanced-and-fourier": advanced_and_fourier,\
        "advanced": advanced_columns,\
        "fourier": fourier_columns,\
        "all_columns": all_columns,
    }


def reduce_df_size(df):
    original_usage = df.memory_usage().sum() / (1024**2)
    for col in df.columns:
        if "fourier.value" in col or \
            "tail" in col or \
            "head" in col or \
            "end" in col or \
            "begin" in col:
            df.drop([col], axis=1, inplace=True)
            continue
        if USE_FLOAT32:
            if "float64" in str(df[col].dtype):
                df[col] = df[col].astype('float32')
        elif "int64" in str(df[col].dtype):
            df[col] = df[col].astype("int8")
    new_usage = df.memory_usage().sum() / (1024**2)
    print(f"Reduced memory sage from {original_usage} to {new_usage}")
    return df

def call_gc():
    for i in range(3):
        for j in range(3):
            gc.collect(j)

@lru_cache(maxsize=5)
def load_datasets():
    datasets = dict()
    for file in glob.glob("MSCPROJDATA/**.parquet.gz", recursive=True):
        if not file.startswith("MSCPROJDATA/n1"):
            print(f"Loading {file}")
            df = pd.read_parquet(file)
            datasets[file] = df
            df["is_encrypted"] = 1 if "encr" in file.lower() else 0
    
            if REDUCE_DF_SIZE:
                df = reduce_df_size(df)

            df = df.sample(frac=1).reset_index(drop=True)
            datasets[file] = df
            call_gc()

    return datasets

_ = load_datasets()


Loading MSCPROJDATA/plaintext.expanded.parquet.gz
Reduced memory sage from 417.6138381958008 to 27.75867462158203
Loading MSCPROJDATA/expanded.pyencrypted_v1.b32.parquet.gz
Reduced memory sage from 417.6138381958008 to 29.300823211669922
Loading MSCPROJDATA/plaintext.combined.parquet.gz
